# Прогнозирование стоимости автомобиля

## 0. Импорт библиотек и настройка 

In [ ]:
# Импорт библиотек

import pandas as pd
import numpy as np
import re
import seaborn as sns
from numpy import percentile
from itertools import combinations
from scipy.stats import ttest_ind
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import f_classif
from seaborn import countplot
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
import warnings
import os

warnings.filterwarnings("ignore")

### Определим функции обработки тренировочного и тестового датасетов
Это необходимо для того ,чтобы привести датасеты к одному виду для дальнейшей оценки

In [ ]:
# Создадим словарь соответствия цветов

rename_color = {
    '040001': 'чёрный',
    'FAFBFB': 'белый',
    'CACECB': 'серебристый',
    '97948F': 'серый',
    '0000CC': 'синий',
    'EE1D19': 'красный',
    '200204': 'коричневый',
    '007F00': 'зелёный',
    'C49648': 'бежевый',
    '22A0F8': 'голубой',
    'DEA522': 'золотистый',
    '660099': 'пурпурный',
    'FFD600': 'жёлтый',
    '4A2197': 'фиолетовый',
    'FF8649': 'оранжевый',
    'FFC0CB': 'розовый'
}
    
# Создадим словарь соответствия трансмиссии
rename_vehicleTr = {
    'AUTOMATIC': 'автоматическая',
    'MECHANICAL': 'механическая',
    'VARIATOR': 'вариатор',
    'ROBOT': 'роботизированная'
}
    
# Создадим словарь соответствия положения ПТС
rename_pts = {
    'ORIGINAL': 1,
    'DUPLICATE': 0
}

rename_pts_rus = {
    'Оригинал': 1,
    'Дубликат': 0
}

# Создадим словарь соответствия положения руля
rename_wheel = {
    'LEFT': 1,
    'RIGHT': 0
}

rename_wheel_rus = {
    'Левый': 1,
    'Правый': 0
}

### Определим функции преобразования суммарного датасета

In [ ]:
# Рассмотрим как распределены числовые признаки в трейне

def train_hist_plot(column_name, df):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    new_series_log = np.log(df[df['train']==1][column_name] + 1)

    ax[0].hist(df[df['train']==1][column_name], rwidth=0.9, alpha=0.7, bins=15)
    ax[0].set_title(column_name)

    ax[1].hist(new_series_log, rwidth=0.9, alpha=0.7, bins=15)
    ax[1].set_title('log of ' + column_name)

    plt.show()
    
def test_hist_plot(column_name, df):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    new_series_log = np.log(df[df['train']==0][column_name] + 1)

    ax[0].hist(df[df['train']==0][column_name], rwidth=0.9, alpha=0.7, bins=15)
    ax[0].set_title(column_name)

    ax[1].hist(new_series_log, rwidth=0.9, alpha=0.7, bins=15)
    ax[1].set_title('log of ' + column_name)

    plt.show()
        
        
# Функция анализа выбросов

def data_outlier(columns_list, df):

    data_out = pd.DataFrame(data = {'name': [], 'count': [], 'min': [], 'mean': [], 'max': [], 
                                    'low_range': [], 'upper_range': [], 'out_count': []
                                   })
    for column_name in columns_list:
        perc25 = percentile(df[column_name], 25)
        perc75 = percentile(df[column_name], 75)
        iqr = perc75 - perc25
        low_range = perc25 - 1.5 * iqr
        upper_range = perc75 + 1.5 * iqr
        out_count = df[df['train']==1][column_name].apply(
            lambda x: None if x < low_range or x > upper_range else x).isna().sum()
        
        to_append = [column_name, df[column_name].count(), round(df[column_name].min(), 2), 
                     round(df[column_name].mean(), 2), round(df[column_name].max(), 2), 
                     round(low_range, 2), round(upper_range, 2), out_count]
        data_out_length = len(data_out)
        data_out.loc[data_out_length] = to_append
        
    return data_out


# Функция для вычисления критерия Стьюдента и записи значимых переменных в новый список
def get_stat_dif(column, stud_math, new_list):
    cols = stud_math.loc[:, column].dropna().unique()
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        t_stud = ttest_ind(stud_math.loc[stud_math.loc[:, column] == comb[0], 'price'].dropna(),
                           stud_math.loc[stud_math.loc[:, column] == comb[1], 'price'].dropna()).pvalue
        t_stud = np.nan_to_num(t_stud)
        if t_stud <= 0.05/len(combinations_all):  # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            new_list.append(column)
            break
            
            
# Функция MAPE для оценки качества работы моделей
def mape(Y_actual,Y_Predicted):
    mape = round(np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100, 2)
    return mape

### Предварительные настройки и загрузка данных
Загрузка тренировочного датасета (готового)
<br>
Загрузка тестового датасета
<br>
Загрузка sample_submission

In [ ]:
# фиксируем RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Загрузка датасетов для kaggle

DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'

df_train = pd.read_csv(DIR_TRAIN + 'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
df_test = pd.read_csv(DIR_TEST + 'test.csv') # тестовый датасет
df_samp = pd.read_csv(DIR_TEST + 'sample_submission.csv') # sample_submissionDIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'

df_train = pd.read_csv(DIR_TRAIN + 'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
df_test = pd.read_csv(DIR_TEST + 'test.csv') # тестовый датасет
df_samp = pd.read_csv(DIR_TEST + 'sample_submission.csv') # sample_submission

In [ ]:
# Загрузка датасетов для локального компьютера

# df2 = pd.read_csv('all_auto_ru_09_09_2020.csv') # тренировочный датасет
# df = pd.read_csv('test_06_new.csv') # тестовый датасет
# df_samp = pd.read_csv('sample_submission.csv') # sample_submission

## 1. Обработка датасетов

### 1.1 Тренировочный датасет

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.nunique()

In [ ]:
# Так как мало значений в полях Состояние, Владение и hidden, удаляем их
# Удалим поле description
df_train = df_train.drop(['Состояние', 'description', 'hidden', 'Владение',
               'Таможня', 'start_date'], axis=1)

# Изменим название поля model на model_name
df_train = df_train.rename(columns={'model':'model_name'})

# Добавим sell_id = 0, так как в тренировочном датасете нет признака id
df_train['sell_id'] = 0

### 1.2 Тестовый датасет

In [ ]:
df_test.head()

In [ ]:
df_test.info()

In [ ]:
df_test.nunique()

In [ ]:
# Так как мало значений в полях complectation_dict и Владение, удаляем их
# Уберем также ненужное поле car_url, description, image, parsing_unixtime, vendor
# Так как в тренировочном датачете удалено поле Состояние, его мы здесь также удалим
# Удалим и столбцы, в которых только 1 уникальное значение

df_test = df_test.drop(['description', 'complectation_dict', 'Владение', 'car_url', 'image', 'parsing_unixtime', 
              'vendor', 'Состояние', 'priceCurrency', 'Таможня'], axis=1)


# Добавим в тестовый датасет признак price = 0, чтобы при объединении датасетов не возникло NaN значение
df_test['price'] = 0

### 1.3 Последовательное сравнение признаков в датасетах и приведение к единому виду

#### Признак BodyType

In [ ]:
display(df_train['bodyType'].nunique())
df_train['bodyType'].value_counts()

In [ ]:
# Выбираем только необходмое название
df_train['bodyType'] = df_train['bodyType'].apply(lambda x: str(x).split(' ')[0].lower())

# Удаляем nan значения
df_train = df_train[df_train['bodyType'] != 'nan']

In [ ]:
display(df_test['bodyType'].nunique())
df_test['bodyType'].value_counts()

In [ ]:
# Выбираем только необходмое название
df_test['bodyType'] = df_test['bodyType'].apply(lambda x: x.split(' ')[0].lower())

#### Признак Brand

In [ ]:
display(df_train['brand'].nunique())
df_train['brand'].value_counts()

In [ ]:
display(df_test['brand'].nunique())
df_test['brand'].value_counts()

#### Признак Color

In [ ]:
display(df_train['color'].nunique())
df_train['color'].value_counts()

In [ ]:
# Приводим через словарь к значениям как в тесте
df_train['color'] = df_train['color'].replace(rename_color)

In [ ]:
display(df_test['color'].nunique())
df_test['color'].value_counts()

#### Признак EngineDisplacement

In [ ]:
df_train['engineDisplacement'].value_counts()

In [ ]:
# Так как поле engineDisplacement заполнено не совсем корректно, взяли данный параметр из name
# Во всех электрокарах заменили данный параметр на 0

df_train['engineDisplacement'] = df_train['name'].astype(str).apply(
    lambda x: re.findall(r"(\d+\.\d+)", x.split('(')[0])).apply(lambda x: 0 if not x else x[0]).astype(float)

In [ ]:
df_test['engineDisplacement'].value_counts()

In [ ]:
# Убираем литры и переводим в тип float
# Заполняем пустые значения на самое популярное - 2.0

df_test['engineDisplacement'] = df_test['engineDisplacement'].apply(
    lambda x: '2.0' if x == ' LTR' else x.split(' ')[0]).astype(float)

#### Признак EnginePower

In [ ]:
df_train['enginePower'].value_counts()

In [ ]:
# Переводим в тип int
df_train['enginePower'] = df_train['enginePower'].astype(int)

In [ ]:
df_test['enginePower'].value_counts()

In [ ]:
# Выбираем только числовое значение
df_test['enginePower'] = df_test['enginePower'].apply(lambda x: x.split(" ")[0]).astype(int)

#### Признак Equipment_dict / Комплектация

In [ ]:
# Смотрим, какая доля null значений. Их слишком много, удаляем столбец
print(round(df_test['equipment_dict'].isna().sum() / df_test['equipment_dict'].count(), 3))

df_test = df_test.drop(['equipment_dict'], axis=1)

In [ ]:
# Так как в тесте удалили столбец equipment_dict, удаляем его здесь тоже
df_train = df_train.drop(['Комплектация'], axis=1)

#### Признак FuelType

In [ ]:
df_train['fuelType'].value_counts()

In [ ]:
df_test['fuelType'].value_counts()

#### Признак ModelDate

In [ ]:
# Приводим к типу int
df_train['modelDate'] = df_train['modelDate'].astype(int)

#### Признак Model_info

In [ ]:
# Удаляем столбец, так как аналогичные данные есть в других столбцах
df_test = df_test.drop(['model_info'],axis=1)

#### Признак Model_name

In [ ]:
df_train['model_name'].value_counts()[:20]

In [ ]:
df_test['model_name'].value_counts()[:20]

#### Признаки Name, Super_gen, vehicleConfiguration

In [ ]:
# Так как значения данного поля содержатся в других полях, удалим его
# Мы уже использовали данное поле
df_train = df_train.drop(['name'], axis=1)
df_test = df_test.drop(['name'], axis=1)

In [ ]:
# Так как значения данного поля содержатся в других полях, удалим его
df_test = df_test.drop(['super_gen'], axis=1)

In [ ]:
# Так как значения данного поля содержатся в других полях, удалим его
df_train = df_train.drop(['vehicleConfiguration'], axis=1)
df_test = df_test.drop(['vehicleConfiguration'], axis=1)

#### Признак NumberOfDoors

In [ ]:
df_train['numberOfDoors'].value_counts()

In [ ]:
# Заменим 0 на самое распространенное - 5, и приведем к типу int

df_train['numberOfDoors'] = df_train['numberOfDoors'].apply(lambda x: 5 if x == 0.0 else x)
df_train['numberOfDoors'] = df_train['numberOfDoors'].astype(int)

In [ ]:
df_test['numberOfDoors'].value_counts()

In [ ]:
# Заменим 0 на самое распространенное - 5, и приведем к типу int

df_test['numberOfDoors'] = df_test['numberOfDoors'].apply(lambda x: 5 if x == 0 else x)
df_test['numberOfDoors'] = df_test['numberOfDoors'].astype(int)

#### Признак VehicleTransmission

In [ ]:
df_train['vehicleTransmission'].value_counts()

In [ ]:
# Заменим поле трансмиссии в трейне
df_train['vehicleTransmission'] = df_train['vehicleTransmission'].replace(rename_vehicleTr)

In [ ]:
df_test['vehicleTransmission'].value_counts()

#### Признак Владельцы

In [ ]:
df_train['Владельцы'].value_counts(dropna=False)

In [ ]:
# Так как есть пустые значения, проверим, как связаны mileage и владельцы,
# чтобы понять, как заполнить поле

print(df_train[df_train['mileage'] == 0]['Владельцы'].value_counts(dropna=False))

# Соответственно это новые авто, заменим null значение на 0, и приведем к типу int
df_train['Владельцы'] = df_train['Владельцы'].fillna(0).astype(int)

In [ ]:
df_test['Владельцы'].value_counts(dropna=False)

In [ ]:
# Приведем к типу int
df_test['Владельцы'] = df_test['Владельцы'].apply(lambda x: x[0]).astype(int)

#### Признак ПТС

In [ ]:
df_train['ПТС'].value_counts(dropna=False)

In [ ]:
# Удалим null значения
# Приведем значения к бинарным значениям через словарь
df_train = df_train[df_train['ПТС'].isna() == False]
df_train['ПТС'] = df_train['ПТС'].replace(rename_pts).astype(int)

In [ ]:
df_test['ПТС'].value_counts(dropna=False)

In [ ]:
# Так как всего одно null значение, заменим его на "Оригинал"
# Приведем значения к бинарным значениям через словарь
df_test['ПТС'] = df_test['ПТС'].fillna('Оригинал')
df_test['ПТС'] = df_test['ПТС'].replace(rename_pts_rus).astype(int)

#### Признак Привод

In [ ]:
df_train['Привод'].value_counts()

In [ ]:
df_test['Привод'].value_counts()

#### Признак Руль

In [ ]:
df_train['Руль'].value_counts()

In [ ]:
# Приведем значения к бинарным значениям через словарь
df_train['Руль'] = df_train['Руль'].replace(rename_wheel).astype(int)

In [ ]:
df_test['Руль'].value_counts()

In [ ]:
# Приведем значения к бинарным значениям через словарь
df_test['Руль'] = df_test['Руль'].replace(rename_wheel_rus).astype(int)

#### Признак Price

In [ ]:
df_train['price'].value_counts()

In [ ]:
# Удалим пустые значения

df_train = df_train[df_train['price'].isna() == False]
df_train['price']= df_train['price'].astype(int)

#### Признак Train
Создадим признак train = 1 для тренировочного датасета и train = 0 для тестового.
Также добавим признак sell_id = 0 для тренировочного датасета

In [ ]:
df_train['train'] = 1
df_test['train'] = 0

In [ ]:
# Сортируем названия колонок

df_train = df_train.reindex(columns=sorted(df_train.columns))
df_test = df_test.reindex(columns=sorted(df_test.columns))

****
# Преобразование и анализ суммарного датасета

In [ ]:
df = pd.concat([df_test, df_train])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.nunique()

In [ ]:
# Определим типы столбцов: числовые, категориальные, бинарные (за исключение столбца train, sell_id они технические)

bin_cols = ['ПТС', 'Руль']
num_cols = ['engineDisplacement', 'enginePower', 'mileage', 'modelDate', 'productionDate', 'Владельцы']
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'model_name', 'numberOfDoors', 
           'vehicleTransmission', 'Привод']
value = ['price']

### Рассмотрим матрицу корреляции для начального удаления сильно скоррелированных признаков

In [ ]:
plt.figure(figsize=(10,10))

sns.heatmap(df[df['train']==1].corr().abs(), annot=True, fmt='.2f', cmap='PuBu')
plt.show()

Так как modelDate и productionDate очень сильно коррелируют, то удалим поле modelDate, productionDate чуть сильнее коррелирует с price.
<br> 
Между enginePower и engineDisplacement тоже сильная корреляция, но пока не будем их трогать

In [ ]:
# df = df.drop(['modelDate', 'engineDisplacement'], axis=1)
df = df.drop(['modelDate'], axis=1)

num_cols.remove('modelDate')
# num_cols.remove('engineDisplacement')

In [ ]:
# Определим новый датафрейм для анализа выбросов признаков
data_out = data_outlier(num_cols + value, df)

## 2.1. Рассмотрим числовые признаки по-отдельности

### EngineDisplacement

In [ ]:
train_hist_plot('engineDisplacement', df)
test_hist_plot('engineDisplacement', df)

При логарифмировании просматривается некоторое улучшение распределения, прологарифмируем признак

In [ ]:
plt.figure(figsize = (10,10))
sns.regplot(data=df[df['train'] == 1], y="price", x="engineDisplacement", fit_reg=False)
plt.show()

In [ ]:
df = df[df['price'] < 35_000_000]

In [ ]:
# Посмотрим на машины с engineDisplacement = 0
# display(df[(df['engineDisplacement'] == 0) & (df['train'] == 1)]['fuelType'].value_counts())


# #Какой engineDisplacement у трейна у электрокаров
# display(df[(df['fuelType'] == 'электро') & (df['train'] == 1)]['engineDisplacement'].value_counts())

# #Какой engineDisplacement у теста у электрокаров
# display(df[(df['fuelType'] == 'электро') & (df['train'] == 0)]['engineDisplacement'].value_counts())

# Соответственно, наше предположение о том, что у электрокаров engineDisplacement = 0 не совпадает с трестовой выборкой
# Заменим в тесте 2.0 на 0

# df.loc[(df['train'] == 0) & (df['fuelType'] == 'электро'), 'engineDisplacement'] = 0.0

In [ ]:
# График выбросов
sns.boxplot(x=df[df['train'] == 1]["engineDisplacement"])
plt.show()

# Выбросы
data_out[data_out['name'] == 'engineDisplacement']

In [ ]:
# Посмотрим как распределена цена у выбросов и невыбросов в трейне
# df[(df['engineDisplacement'] > 3.85) & (df['train'] == 1)]['price_log'].hist()
# plt.show()

# df[(df['engineDisplacement'] <= 3.85) & (df['train'] == 1)]['price_log'].hist()
# plt.show()

Цена распределена примерно одинакого, ничего с выбросами делать пока не будем

In [ ]:
# df = df[(df['train'] == 0) | ((df['train'] == 1) & (df['engineDisplacement'] > 0) & (df['engineDisplacement'] < 3.85))]

In [ ]:
df['engineDisp_log'] = np.log(df['engineDisplacement'] + 1)

#### Вывод:
    Создали новыйй признак как логарифм от engineDisplacement
    Выбросов слишком много, пока ничего делать с ними не будем

### EnginePower

In [ ]:
train_hist_plot('enginePower', df)
test_hist_plot('enginePower', df)

После логарифмирования видим более симметричное распределение, прологарифмируем признак

In [ ]:
plt.figure(figsize = (10,10))
sns.regplot(data=df[df['train'] == 1], y="price", x="enginePower", fit_reg=False)
plt.show()

In [ ]:
# График выбросов
sns.boxplot(x=df[df['train'] == 1]["enginePower"])
plt.show()

# Выбросы
data_out[data_out['name'] == 'enginePower']

Ничего с выбросами делать пока не будем

In [ ]:
# df = df[(df['train'] == 0) | ((df['train'] == 1) & (df['enginePower'] > 0) & (df['enginePower'] < 312.5))]

In [ ]:
df['enginePower_log'] = np.log(df['enginePower'] + 1)

#### Вывод:
    Создали новыйй признак как логарифм от enginePower
    Выбросов слишком много, пока ничего делать с ними не будем

### Mileage

In [ ]:
train_hist_plot('mileage', df)
test_hist_plot('mileage', df)

Улучшения не видно, также сильно выделяется модели с 0-ым пробегом.
<br>
Посмотрим, есть ли в тесте модели с 0-ым пробегом.

In [ ]:
plt.figure(figsize = (10,10))
sns.regplot(data=df[df['train'] == 1], y="price", x="mileage", fit_reg=False)
plt.show()

In [ ]:
# По предварительной обработке видно, что это новые маишны с владельцами = 0. В тесте нет владельцев = 0 и mileage = 0.
# Сделаем новую бинарную переменную mileage_0
df['mileage_0'] = df['mileage'].apply(lambda x: 1 if x == 0 else 0)

In [ ]:
# График выбросов
sns.boxplot(x=df[df['train'] == 1]["mileage"])
plt.show()

# Выбросы
data_out[data_out['name'] == 'mileage']

Ничего с выбросами делать пока не будем

In [ ]:
# df = df[(df['train'] == 0) | ((df['train'] == 1) & (df['mileage'] < 395000))]

#### Вывод:
    Выбросов слишком много, пока ничего делать с ними не будем

### ProductionDate

In [ ]:
train_hist_plot('productionDate', df)
test_hist_plot('productionDate', df)

Оставим как есть

In [ ]:
# Теперь посмотрим, как изменится график распределения если убрать даты до 1991
train_hist_plot('productionDate', df[df['productionDate'] > 1990])
test_hist_plot('productionDate', df[df['productionDate'] > 1990])

Так как после удаления машин до 1990 года, количество выбросов уменьшилось,
<br>
не будем их использовать в тренировке

In [ ]:
plt.figure(figsize = (10,10))
sns.regplot(data=df[df['train'] == 1], y="price", x="productionDate", fit_reg=False)
plt.show()

In [ ]:
df = df[((df['productionDate'] > 1990) & (df['productionDate'] < 2020) & (df['train'] == 1)) | (df['train'] == 0)]
# df = df[((df['productionDate'] > 1990) & (df['train'] == 1)) | (df['train'] == 0)]

#### Вывод:
    Не будем использовать данные машин по 1990 год в трейне
    Много машин в трейне 2020 года, портят распределение цены, но немного в тесте, возможно удалим потом

### Владельцы

In [ ]:
train_hist_plot('Владельцы', df)
test_hist_plot('Владельцы', df)

In [ ]:
plt.figure(figsize = (10,10))
sns.regplot(data=df[df['train'] == 1], y="price", x="Владельцы", fit_reg=False)
plt.show()

In [ ]:
# График выбросов
sns.boxplot(x=df[df['train'] == 1]["Владельцы"])
plt.show()

# Выбросы
data_out[data_out['name'] == 'Владельцы']

#### Вывод:
     Оставляем все как есть

### Price

In [ ]:
train_hist_plot('price', df)

После логарифмирования видим более симметричное распределение, прологарифмируем признак

In [ ]:
# Изменим цену для проверки того, что цена в полученном датасете может сильно отличаться,
# так как данный датасет был взят достаточно давно, а курс доллара сильно изменился за это время

df['price'] = round(df['price']*1,4)

In [ ]:
df['price_log'] = np.log(df['price'] + 1)


## 2.2. Создание новых числовых переменных

In [ ]:
df['mileage_for_date'] = round(df['mileage'] / (2021 - df['productionDate']), 2)
df['mileage_for_owner'] = round(df['Владельцы'] / (2021 - df['productionDate']), 2)

#### Удаляем признаки 

In [ ]:
# Удаляем признаки engineDisplacement и enginePower, так как будем использовать их логарим

# df = df.drop(['enginePower'], axis=1)
df = df.drop(['engineDisplacement', 'enginePower'], axis=1)

#### Создадим новые списки переменны

In [ ]:
df.columns

In [ ]:
num_cols_new = ['engineDisp_log', 'enginePower_log', 'mileage', 'productionDate', 'Владельцы',
                'mileage_for_date', 'mileage_for_owner']
# num_cols_new = ['enginePower_log', 'mileage', 'productionDate', 'Владельцы',
#                 'mileage_for_date', 'mileage_for_owner']

bin_cols_new = ['ПТС', 'Руль', 'mileage_0']
# bin_cols_new = ['ПТС', 'Руль']

## 2.3. Значимость числовы и бинарных переменных

In [ ]:
df_for_imp = df.copy()

# рассмотрим значимость переменных
label_encoder = LabelEncoder()

imp_num = pd.Series(f_classif(df_for_imp[df_for_imp['train']==1][bin_cols_new + num_cols_new], 
                              df_for_imp[df_for_imp['train']==1]['price_log'])[0], index=bin_cols_new + num_cols_new)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

#### Вывод:
    Видно, что созданные переменные имеют большее влияние на цену, чем бинарные признаки "ПТС" и "Руль".
    Самым значимым является признак productionDate, а наименее значимым тип руля

### Рассмотрим корреляцию получившихся числовых переменных

In [ ]:
plt.figure(figsize=(10,10))

sns.heatmap(df[df['train']==1][num_cols_new + bin_cols_new + ['price_log']].corr().abs(), annot=True, fmt='.2f', cmap='PuBu')
plt.show()

In [ ]:
# Удалим сильно скоррелированный признак engineDisp_log
# df = df.drop(['engineDisp_log'], axis=1)

# num_cols_new.remove('engineDisp_log')

## 2.4. Категориальные признаки

In [ ]:
display(df[df['train'] == 1][cat_cols].nunique())
display(df[df['train'] == 0][cat_cols].nunique())

Как видно из данных, по некоторым признакам в трейне количество уникальных значений превышает значение в тесте.
<br>
Просмотрим признаки по-отдельности

### Brand

In [ ]:
# Проверим, что все типы совпадают в трейне и тесте

sorted(set(df[df['train'] == 1]['brand'])) == sorted(set(df[df['train'] == 0]['brand']))

In [ ]:
# Посмотрим как распрелено их количество

plt.figure(figsize=(5,5))

sns.countplot(y="brand", data=df[df['train'] == 1], order = df[df['train'] == 1]['brand'].value_counts().index)
plt.show()

plt.figure(figsize=(5,5))
sns.countplot(y="brand", data=df[df['train'] == 0], order = df[df['train'] == 0]['brand'].value_counts().index)
plt.show()

In [ ]:
# Рассмотрим как отличаются цены на бренды не из теста
brand_list = list(set(df[df['train'] == 0]['brand']))
df_brand = df[(df['train'] == 1) & (df['brand'].isin(brand_list) == False)]


plt.figure(figsize = (20,10))
sns.boxplot(y='brand', x="price",data=df[(df['train'] == 1) & (df['brand'].isin(brand_list))])
plt.show()

plt.figure(figsize = (20,10))
sns.boxplot(y='brand', x="price",data=df_brand)
plt.show()

#### Вывод:
    Их очень много, ничего с ними делать не будем
    Не уменьшаем количество брендов, так как это ухудшает качество модели

### BodyType

In [ ]:
# Проверим, что все типы совпадают в трейне и тесте

sorted(set(df[df['train'] == 1]['bodyType'])) == sorted(set(df[df['train'] == 0]['bodyType']))

In [ ]:
# Посмотрим, как распределены основные типы машин

plt.figure(figsize=(5,5))

sns.countplot(y="bodyType", data=df[df['train'] == 1], order = df[df['train'] == 1]['bodyType'].value_counts().index)
plt.show()

plt.figure(figsize=(5,5))
sns.countplot(y="bodyType", data=df[df['train'] == 0], order = df[df['train'] == 0]['bodyType'].value_counts().index)
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(y='bodyType', x="price",data=df[df['train'] == 1], order = df['bodyType'].value_counts().index)
plt.show()

In [ ]:
# bodyType_top10 = df['bodyType'].value_counts().index[:12]
# df['bodyType'] = df['bodyType'].apply(lambda x: 'other' if not x in bodyType_top10 else x)

Очень есть типы bodyType с очень малым количеством значений.
<br>
Уменьшение числа bodyType ведет к увеличению MAPE

### Color

In [ ]:
# Посмотрим как распрелено их количество

plt.figure(figsize=(5,5))

sns.countplot(y="color", data=df[df['train'] == 1], order = df[df['train'] == 1]['color'].value_counts().index)
plt.show()

plt.figure(figsize=(5,5))
sns.countplot(y="color", data=df[df['train'] == 0], order = df[df['train'] == 0]['color'].value_counts().index)
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(y='color', x="price",data=df[df['train'] == 1], order = df['color'].value_counts().index)
plt.show()

In [ ]:
# color_top10 = df['color'].value_counts().index[:10]
# df['color'] = df['color'].apply(lambda x: 'other' if not x in color_top10 else x)

#### Вывод:
    Не уменьшаем количество цветов, так как это ухудшает качество модели

### FuelType

In [ ]:
# Посмотрим как распрелено их количество

plt.figure(figsize=(5,5))

sns.countplot(y="fuelType", data=df[df['train'] == 1], order = df[df['train'] == 1]['fuelType'].value_counts().index)
plt.show()

plt.figure(figsize=(5,5))
sns.countplot(y="fuelType", data=df[df['train'] == 0], order = df[df['train'] == 0]['fuelType'].value_counts().index)
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(y='fuelType', x="price",data=df[df['train'] == 1], order = df['fuelType'].value_counts().index)
plt.show()

#### Вывод:
    Оставим все как есть

### Model_name
Как видно из данных, параметр model_name содержит слишком много уникальных значений,
однако, надо проверить значимость переменных чтобы понять, стоит ли удалять

In [ ]:
model_name_test = list(set(df[df['train'] == 0]['model_name']))
df[(df['train'] == 1) & (df['model_name'].isin(model_name_test) == False)].shape[0]

Так как количество моделей из трейна, которых нет в тесте, занимают большую долю из датафрейма,
нельзя просто удалить данные из трейна. Скорее всего просто удалим этот параметр

### NumberOfDoors

In [ ]:
# Посмотрим как распрелено их количество

plt.figure(figsize=(5,5))

sns.countplot(y="numberOfDoors", data=df[df['train'] == 1], order = df[df['train'] == 1]['numberOfDoors'].value_counts().index)
plt.show()

plt.figure(figsize=(5,5))
sns.countplot(y="numberOfDoors", data=df[df['train'] == 0], order = df[df['train'] == 0]['numberOfDoors'].value_counts().index)
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x='numberOfDoors', y="price",data=df[df['train'] == 1])
plt.show()

### VehicleTransmission

In [ ]:
# Посмотрим как распрелено их количество

plt.figure(figsize=(5,5))

sns.countplot(y="vehicleTransmission", data=df[df['train'] == 1], 
              order = df[df['train'] == 1]['vehicleTransmission'].value_counts().index)
plt.show()

plt.figure(figsize=(5,5))
sns.countplot(y="vehicleTransmission", data=df[df['train'] == 0], 
              order = df[df['train'] == 0]['vehicleTransmission'].value_counts().index)
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x='vehicleTransmission', y="price",data=df[df['train'] == 1])
plt.show()

### Привод

In [ ]:
# Посмотрим как распрелено их количество

plt.figure(figsize=(5,5))

sns.countplot(y="Привод", data=df[df['train'] == 1], 
              order = df[df['train'] == 1]['Привод'].value_counts().index)
plt.show()

plt.figure(figsize=(5,5))
sns.countplot(y="Привод", data=df[df['train'] == 0], 
              order = df[df['train'] == 0]['Привод'].value_counts().index)
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
sns.boxplot(x='Привод', y="price",data=df[df['train'] == 1])
plt.show()

## 2.5. Значимость категориальных переменных

In [ ]:
label_encoder = LabelEncoder()

for column in cat_cols:
    df_for_imp[column] = label_encoder.fit_transform(df_for_imp[column])

imp_cat = pd.Series(mutual_info_classif(df_for_imp[df_for_imp['train']==1][cat_cols], 
                                        df_for_imp[df_for_imp['train']==1]['price'], discrete_features = True), index=cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

plt.show()

#### Вывод:
    Видно, что model_name является самой значимой переменной для price. Нельзя просто удалить этот признакю
    Самым наименее значимым является признак fuelType

## Тест Стьюдента
Определим, какие из категориальных переменных являются значимыми для price

In [ ]:
object_list_new = []

for col in cat_cols:
    get_stat_dif(col, df[df['train'] == 1], object_list_new)

#### Вывод:
    Мы определили, что все категориальные переменные значимы

## 2.6. Преобразование категориальных переменных

In [ ]:
# Создадим копию dataframe

df_output = df.copy()

Для категориальных переменных, у которых много уникальных значений будем использоваться label_encoder, чтобы перевести тип str к типу int

In [ ]:
label = LabelEncoder()

# df_output['brand_label'] = label.fit_transform(df['brand'])
# df_output['model_name_label'] = label.fit_transform(df['model_name'])


# Удаляем ненужный признак model_name
df_output = df_output.drop(['brand', 'model_name'], axis=1)


display(df_output.head())

In [ ]:
# Создание dummies признаков
# df_output = pd.get_dummies(df_output, columns=['bodyType', 'brand', 'color', 'fuelType', 'numberOfDoors', 
#                                         'vehicleTransmission', 'Привод'])

df_output = pd.get_dummies(df_output, columns=['bodyType', 'color', 'fuelType', 'numberOfDoors', 
                                        'vehicleTransmission', 'Привод'])

#### Создадим новый список категориальных переменных

In [ ]:
cat_cols_new = [x for x in list(df_output.columns) if x not in num_cols_new and 
                x not in bin_cols_new and x not in ['price', 'price_log', 'train', 'sell_id']]

In [ ]:
df_output.shape

In [ ]:
display(bin_cols_new, num_cols_new, cat_cols_new)

## 3. Подготовка данных для модели

In [ ]:
df_X_train = df_output[df_output['train'] == 1]
df_X_test = df_output[df_output['train'] == 0]

X_bin_train = df_X_train[bin_cols_new].values
X_bin_test = df_X_test[bin_cols_new].values

X_cat_train = df_X_train[cat_cols_new].values
X_cat_test = df_X_test[cat_cols_new].values

robust = RobustScaler().fit(df_X_train[num_cols_new].values)
X_num_train = robust.transform(df_X_train[num_cols_new].values)
X_num_test = robust.transform(df_X_test[num_cols_new].values)


# Объединяем данные
X = np.hstack([X_bin_train, X_num_train, X_cat_train])
X_sub = np.hstack([X_bin_test, X_cat_test, X_num_test])
Y = df_X_train['price_log'].values
# test_val = np.hstack([X_bin_test, X_num_test, X_cat_test])

## 4. Моделирование и оценка эффективности

### Алгоритм RandomForestRegressor

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_SEED, shuffle = True)

model = RandomForestRegressor(random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

# Преобразуем Y_test, y_pred к exp значениям для оценки MAPE

y_pred = np.round(np.exp(y_pred) - 1)
Y_test = np.round(np.exp(Y_test) - 1)

# Вывод результата MAPE
mape(Y_test, y_pred)

#### Выбираем лучшие параметры

In [ ]:
parameters = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [1, 2, 5]
}


model = RandomForestRegressor(random_state=RANDOM_SEED)
grid_rand = GridSearchCV(estimator=model, param_grid=parameters, 
                         scoring='neg_mean_squared_error', n_jobs=-1, cv=5)

grid_rand.fit(X_train, Y_train)
grid_rand.best_params_

#### Посмотрим как изменится MAPE

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_SEED, shuffle = True)

model = RandomForestRegressor(max_depth=20, min_samples_split=5, n_estimators=200, random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

y_pred = np.round(np.exp(y_pred) - 1)
Y_test = np.round(np.exp(Y_test) - 1)

mape(Y_test, y_pred)

### Алгоритм KNeighborsRegressor

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_SEED, shuffle = True)

model = KNeighborsRegressor()

# Обучаем модель на тестовом наборе данных
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

# Преобразуем Y_test, y_pred к exp значениям для оценки MAPE

y_pred = np.round(np.exp(y_pred) - 1)
Y_test = np.round(np.exp(Y_test) - 1)

# Вывод результата MAPE
mape(Y_test, y_pred)

#### Выбираем лучшие параметры

In [ ]:
parameters = {
    'n_neighbors': [3, 5, 10],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40]
}

model = KNeighborsRegressor()
grid_rand = GridSearchCV(estimator=model, param_grid=parameters, 
                         scoring='neg_mean_squared_error', n_jobs=-1, cv=5)

grid_rand.fit(X_train, Y_train)
grid_rand.best_params_

#### Посмотрим как изменится MAPE

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_SEED, shuffle = True)

model = KNeighborsRegressor(algorithm='brute', leaf_size=10, n_neighbors=10)

# Обучаем модель на тестовом наборе данных
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

# Преобразуем Y_test, y_pred к exp значениям для оценки MAPE

y_pred = np.round(np.exp(y_pred) - 1)
Y_test = np.round(np.exp(Y_test) - 1)

# Вывод результата MAPE
mape(Y_test, y_pred)

### Алгоритм GradientBoostingRegressor

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_SEED, shuffle = True)

model = GradientBoostingRegressor(random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

# Преобразуем Y_test, y_pred к exp значениям для оценки MAPE

y_pred = np.round(np.exp(y_pred) - 1)
Y_test = np.round(np.exp(Y_test) - 1)

# Вывод результата MAPE
mape(Y_test, y_pred)

#### Выбираем лучшие параметры

In [ ]:
parameters = {
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [50, 100, 200],
    'min_samples_split': [1, 2, 5],
}


model = GradientBoostingRegressor(random_state=RANDOM_SEED)
grid_rand = GridSearchCV(estimator=model, param_grid=parameters, 
                         scoring='neg_mean_squared_error', n_jobs=-1, cv=5)

grid_rand.fit(X_train, Y_train)
grid_rand.best_params_

#### Посмотрим как изменится MAPE

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_SEED, shuffle = True)

model = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=5, n_estimators=200, random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

# Преобразуем Y_test, y_pred к exp значениям для оценки MAPE

y_pred = np.round(np.exp(y_pred) - 1)
Y_test = np.round(np.exp(Y_test) - 1)

# Вывод результата MAPE
mape(Y_test, y_pred)

### Алгоритм StackingRegressor

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=RANDOM_SEED, shuffle = True)

estimators = [
    ('rfr', RandomForestRegressor(random_state=RANDOM_SEED)),
    ('gbr', GradientBoostingRegressor(learning_rate=0.1, min_samples_split=5, n_estimators=200, random_state=RANDOM_SEED))
]


reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(max_depth=20, min_samples_split=5, n_estimators=200, random_state=RANDOM_SEED)

)

# Обучаем модель на тестовом наборе данных
reg.fit(X_train, Y_train)
y_pred = reg.predict(X_test)

y_pred = np.round(np.exp(y_pred) - 1)
Y_test = np.round(np.exp(Y_test) - 1)

# Вывод результата MAPE
mape(Y_test, y_pred)

## 5. Submission

In [ ]:
predict_submission = np.round(np.exp(reg.predict(X_sub)) - 1).astype(int)

df_samp['price'] = predict_submission
df_samp.to_csv(f'submission_TimeryaevaM.csv', index=False)
df_samp.head(10)